In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.signal import hamming

"""This function creates the lattice points for the all the images in a single go
and stacks them in 3D"""
def lattice(parms,nxx=20, nyy=20):
  a1 = parms[0]
  a2 = parms[1]
  phi = parms[2]
  
  nx,ny = np.meshgrid(np.arange(nxx), np.arange(nyy))
  values = (nx.ravel(),ny.ravel())
  values = np.array(values).T
  nxx = values[:,0]
  nyy = values[:,1]
  j = a1.shape[0]
  atom_pos = np.zeros((j,400,2))
  for i in range(0,j):

      x_ind = nxx * a1[i] + nyy * a2[i] * np.cos(phi[i])
      y_ind = nyy * a2[i] * np.sin(phi[i])
      atom_pos[i,:,0] = x_ind
      atom_pos[i,:,1] = y_ind
  
  return atom_pos


"""This function distorts the positions of all the atoms in the lattice with
standard deviation 30% of the lattice parameter a1"""
def distortions(atom_pos, parms):
  j = atom_pos.shape[0]
  a1 = parms[0]
  a2 = parms[1]
  atom_pos_dis = np.zeros((j,400,2))
  for i in range(j):
    x_dis = np.random.normal(loc = 0.0, scale = 0.03*a1[i] ,size = [400,1])
    y_dis = np.random.normal(loc = 0.0, scale = 0.03*a2[i] ,size = [400,1])

    dis = np.concatenate((x_dis,y_dis),axis=1)
    atom_pos_dis[i] = atom_pos[i] + dis
  
  return atom_pos_dis


def atom_to_image(atom_pos_dis,img_dim = 1024):
  j = atom_pos_dis.shape[0]
  image_atoms = np.zeros([j,img_dim,img_dim])
    
  for i in range(0,j):
      max_x = np.max(atom_pos_dis[i,:,0])
      max_y = np.max(atom_pos_dis[i,:,1])

      min_x = np.min(atom_pos_dis[i,:,0])
      min_y = np.min(atom_pos_dis[i,:,1])



      x1,y1 = atom_pos_dis[i,:,0], atom_pos_dis[i,:,1]
      x_img = ((x1 - min_x)/(max_x - min_x) * (img_dim-1))       
      y_img = ((y1-min_y)/(max_y - min_y) * (img_dim-1))

      x_img = x_img.astype(int)
      y_img = y_img.astype(int)  
      image_atoms[i,x_img, y_img]=1E6           


  return image_atoms

def convolve_atomic_image(image_atoms, sigma = 6):
  j = image_atoms.shape[0]
  con_img = np.zeros([j,1024,1024])
  for i in range(0,j):
      con_img[i] = gaussian_filter(image_atoms[i],sigma,order = 0)
  return con_img
def crop(convolved_img):
    crop_img = convolved_img[:,350:700,350:700]
    return crop_img

def fft(crop_img):
    img_ffts=[]
    fft_win_size =64
    j = crop_img.shape[0]
    for j in range(0,j):
        
        n = crop_img.shape[1]
        h = hamming(n) 
        ham2d = np.sqrt(np.outer(h,h))
        img_windowed = np.copy(crop_img[j])
        img_windowed *= ham2d 
        img_fft = np.fft.fftshift(np.fft.fft2(img_windowed))
        img_fft = img_fft[crop_img.shape[1]//2 - fft_win_size:crop_img.shape[1]//2+fft_win_size,
                                     crop_img.shape[1]//2 - fft_win_size:crop_img.shape[1]//2+fft_win_size]
        img_ffts.append(img_fft)
        
    return np.array(np.sqrt(np.abs(img_ffts)))
  
  
  


In [2]:
"""Debugged"""
import numpy as np
from sklearn.utils import shuffle

"""Generates 400 lattice parameters for each type of bravis lattice,
labels and shuffles them and output 100 data points when called"""

def generate():
#oblique   a1 != a2, phi!=90 Label = 0
    bond_len = np.random.uniform(low = [0.8,0.8], high = [2.0,2.2],size = [800,2])
    diff = np.abs(((bond_len[:,0]-bond_len[:,1])*100)/bond_len[:,1])

    bond_len = bond_len[diff > 20.0,:]
    bond_len = bond_len[0:400,:]
    phi = np.zeros((400,1))
    phi[0:300] = np.random.uniform(low = [0.0], high = [((55.0/180)*np.pi)],size = [300,1])
    phi[300:400] = np.random.uniform(low = [((65.0/180)*np.pi)], high = [((90.0/180)*np.pi)],size = [100,1])

    parms_obl = np.concatenate((bond_len,phi),axis = 1)
    
    #square  a1 = a2, phi = 90, Label = 1
    parms_sq = np.zeros([400,3])
    parms_sq[:,0] = np.random.uniform(low = 0.8, high = 2.0,size = 400)
    parms_sq[:,1] = parms_sq[:,0]
    parms_sq[:,2] = np.pi/2
    
    #Hexagonal a1 = a2, phi = 60 Label = 2
    parms_hex = np.zeros([400,3])
    parms_hex[:,0] = np.random.uniform(low = 0.8, high = 2.0,size = 400)
    parms_hex[:,1] = parms_hex[:,0]
    parms_hex[:,2] = (np.pi/3)
    
    #rectangular a1 != a2, phi = 90 Label =3
    parms_rec = np.random.uniform(low = [0.8,0.8,(np.pi/2)], high = [2.0,2.2,(np.pi/2)],size = [800,3])
    diff_rec = np.abs(((parms_rec[:,0]-parms_rec[:,1])*100)/parms_rec[:,1])
    
    parms_rec = parms_rec[diff_rec > 20.0,:]
    parms_rec = parms_rec[0:400,:]
    
    
    #centered  a1 = a2, phi != 90, Label = 4
    bond_len = np.zeros([400,2])
    bond_len[:,0] = np.random.uniform(low = 0.8, high = 2.0,size = 400)
    bond_len[:,1] = bond_len[:,0]
    phi = np.zeros((400,1))
    phi[0:300] = np.random.uniform(low = [0.0], high = [((55.0/180)*np.pi)],size = [300,1])
    phi[300:400] = np.random.uniform(low = [((65.0/180)*np.pi)], high = [((90.0/180)*np.pi)],size = [100,1])

    parms_cen = np.concatenate((bond_len,phi),axis = 1)
    
    parameters = np.concatenate((parms_obl,parms_sq,parms_hex,parms_rec, parms_cen), axis=0 )
    label = np.zeros([2000,1])
    
    label[401:801,0]=1
    label[801:1201,0]=2
    label[1201:1601,0]=3
    label[1601:2001,0]=4
    
    parameters,label = shuffle(parameters, label)
    parameters = parameters.T
    
    return parameters ,label
#     for i in range(0,10):
#         parameters_list = parameters[:,i*100:(i+1)*100]
#         labels_list = label[i*100:(i+1)*100,0]
#         yield parameters_list, labels_list

In [3]:
def images_final(parms):
  atom_pos = lattice(parms)
  atom_pos_dis = distortions(atom_pos, parms)
  image_atoms = atom_to_image(atom_pos_dis)
  con_img = convolve_atomic_image(image_atoms)
  crop_img = crop(con_img)
  img_final = fft(crop_img)

  return img_final

In [5]:
import numpy as np
import torch, torchvision
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

parameters_list, labels_list = generate()
print(parameters_list)
data = images_final(parameters_list)
#data = data.astype(np.float32)

[[1.31279581 1.73698235 0.92282981 ... 1.25397022 1.37216032 1.35472902]
 [1.31279581 1.07946968 1.80255077 ... 1.25397022 1.37216032 1.35472902]
 [1.57079633 1.57079633 1.57079633 ... 1.57079633 1.57079633 1.04719755]]


In [16]:

import os
    

In [17]:
from scipy import  ndimage

def ZoomInterpol(FFT_image, zoom_factor=2, interpol_factor=1):
    """Given a  2D FFT , return a 2D FFT with a zoom factor and interpolation if desired
    Parameters
    ----------
    FFT_image : 2D numpy array
        Numpy matrix of size (m,m). 2D FFT.
    zoom_factor: integer 
        (optional, default=1). Factor by which to magnify the FFT.
    interpol_factor: integer (optional, default = 1). Factor by which to increase the size of the image by interpolation.
        Array of size (nx2), where n is the number of classes as defined in the class dictionary

    Returns
    -------

    2D Numpy array 
        Numpy array that has been zoomed and inteprolated by the requested amounts.

    """

    zoom_size = (FFT_image.shape[0] / zoom_factor) / 2
    window_size_x = FFT_image.shape[0]
    window_size_y = FFT_image.shape[1]

    if np.mod(FFT_image.shape[0] / zoom_factor, 2) == 0:
        F2_zoomed = FFT_image[int(window_size_x / 2 - zoom_size):int(window_size_x / 2 + zoom_size),
                    int(window_size_y / 2 - zoom_size):int(window_size_y / 2 + zoom_size)]
    else:
        F2_zoomed = FFT_image[int(window_size_x / 2 - zoom_size):int(window_size_x / 2 + 1 + zoom_size),
                    int(window_size_y / 2 - zoom_size):int(window_size_y / 2 + 1 + zoom_size)]

    return ndimage.zoom(F2_zoomed, interpol_factor)

In [18]:
z = 0
for i,j in zip(data,labels_list):
   # Create directory
    i = ZoomInterpol(i)
    dirName = "new_data"+"/train/"+str(j)
    try:
    # Create target Directory
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")
    plt.imsave(dirName + "/"+str(z)+".png" ,i, format='png')
    z+=1
        

Directory  new_data/train/[4.]  Created 
Directory  new_data/train/[3.]  Created 
Directory  new_data/train/[1.]  Created 
Directory  new_data/train/[0.]  Created 
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  Created 
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exi

Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  ne

Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  ne

Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  ne

Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  ne

Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  ne

Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  ne

Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  ne

Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  ne

Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[0.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[3.]  already exists
Directory  new_data/train/[2.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[1.]  already exists
Directory  new_data/train/[4.]  already exists
Directory  ne

In [33]:
label

NameError: name 'label' is not defined